In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=5b1c483f64e15496cf4d154279a0ef3c119c959348b275eb8a10266e0a7371ab
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()
spark = SparkSession.builder.getOrCreate()



In [ ]:
display(spark.read.csv("DallasCouncilVoters.csv.gz", inferSchema=True, header=True).show(4))
display(spark.read.csv("DallasCouncilVotes.csv.gz", inferSchema=True, header=True).show(4))

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
+----------+-------------+-------------------+
only showing top 4 rows



None

+----------+------------------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+
|      DATE|AGENDA_ITEM_NUMBER|ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|
+----------+------------------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+
|02/08/2017|                 1|   AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|
|02/08/2017|                 1|   AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|
|02/08/2017|                 1|   AGENDA|      15|        Mayor|Michael S. Rawlings|      N/A|  NO ACTION NEED

None

In [ ]:
# StructField("name", dtype, nullable=True)
peopleSchema = StructType([StructField("name", StringType(), True ), StructField("age", IntegerType(), True), StructField("city", StringType(), True)])
people_df = spark.read.format("csv").load(name="voterdata.csv", schema=peopleSchema) # อันนี้ใส่ Schema มั่วๆเอา

# Read csv
sdf = spark.read.csv("voterdata.csv", inferSchema=True, header=True)

print("Check Schema: Notice that the column 'AGENDA_ITEM_NUMBER' is string.")
display(sdf.printSchema()); print("\n")

# Change AGENDA_ITEM_NUMBER into integer
sdf = sdf.withColumn("AGENDA_ITEM_NUMBER", sdf["AGENDA_ITEM_NUMBER"].cast("integer"))

# Add column
sdf = sdf.withColumn("ITEM+100", sdf["AGENDA_ITEM_NUMBER"]+100)
display(sdf.show(3))

# Drop column
sdf = sdf.drop(sdf["AGENDA_ITEM_NUMBER"])
display(sdf.show(3))

# Count number rows
display(sdf.count())

Check Schema: Notice that the column 'AGENDA_ITEM_NUMBER' is string.
root
 |-- DATE: string (nullable = true)
 |-- AGENDA_ITEM_NUMBER: string (nullable = true)
 |-- ITEM_TYPE: string (nullable = true)
 |-- DISTRICT: integer (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER NAME: string (nullable = true)
 |-- VOTE CAST: string (nullable = true)
 |-- FINAL ACTION TAKEN: string (nullable = true)
 |-- AGENDA ITEM DESCRIPTION: string (nullable = true)
 |-- AGENDA_ID: string (nullable = true)
 |-- VOTE_ID: string (nullable = true)



None



+----------+------------------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|      DATE|AGENDA_ITEM_NUMBER|ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|ITEM+100|
+----------+------------------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|02/08/2017|                 1|   AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|                 1|   AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|                 1|   AGENDA|      15|        Mayor

None

+----------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|      DATE|ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|ITEM+100|
+----------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|02/08/2017|   AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|   AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|   AGENDA|      15|        Mayor|Michael S. Rawlings|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|  

None

7924

In [ ]:
# Load csv file from .gz (Winrar) file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('AA_DFW_2018_Departures_Short.csv.gz')
display(aa_dfw_df.show(4))

# ไม่เห็นจำเป็นต้องเขียนยาวๆ โง่ๆ
display(spark.read.csv("AA_DFW_2018_Departures_Short.csv.gz", header=True).show(4))
sdf = spark.read.csv("AA_DFW_2018_Departures_Short.csv.gz", inferSchema=True, header=True)


# import string function
import pyspark.sql.functions as F
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
display(aa_dfw_df.show(5))

# Add tables to catalog
aa_dfw_df.createOrReplaceTempView("flights")
sdf.createOrReplaceTempView("voters")
display(spark.catalog.listTables())

# Can be used with sql query
tested_df = spark.sql("SELECT * FROM flights WHERE airport = 'hnl'")
tested_df.show(4)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2017|         0005|                HNL|                          537|
|       01/01/2017|         0007|                OGG|                          498|
|       01/01/2017|         0037|                SFO|                          241|
|       01/01/2017|         0043|                DTW|                          134|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 4 rows



None

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2017|         0005|                HNL|                          537|
|       01/01/2017|         0007|                OGG|                          498|
|       01/01/2017|         0037|                SFO|                          241|
|       01/01/2017|         0043|                DTW|                          134|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 4 rows



None

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
+-----------------+-------------+-----------------------------+-------+
only showing top 5 rows



None

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='voters', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0123|                          529|    hnl|
|       01/02/2017|         0005|                          537|    hnl|
|       01/02/2017|         0123|                          532|    hnl|
+-----------------+-------------+-----------------------------+-------+
only showing top 4 rows



In [ ]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet("AA_DFW_ALL.parquet", header=True)
flights_df = flights_df.withColumnRenamed("_c3", "flight_duration")

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 151


In [ ]:
### Working with Parquet (No data provided.  These are just instructions.)

# Read Parquet 
df = spark.read.format("parquet").load("filename.parquet")
df = spark.read.parquet("filename.parquet")

# Write Parquet
df.write.format("parquet").save("filename.parquet")
df.write.parquet("filename.parquet")

# Add tables to catalog to be used for sql query
df.createOrReplaceTempView("Table_name")
filtered_df = spark.sql("SELECT * FROM Table_name WHERE col_1 < 100")


In [ ]:
### Examples with Parquet ###
df1 = spark.read.csv("AA_DFW_2017_Departures_Short.csv.gz")
df2 = spark.read.csv("AA_DFW_2018_Departures_Short.csv.gz")

print("df1 and df2 are Spark DataFrame and look like below:")
display(df1.show(3))
display(df2.show(3)); print("\n")

# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count()); print("\n")

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

df1 and df2 are Spark DataFrame and look like below:
+-----------------+-------------+-------------------+--------------------+
|              _c0|          _c1|                _c2|                 _c3|
+-----------------+-------------+-------------------+--------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed ti...|
|       01/01/2017|         0005|                HNL|                 537|
|       01/01/2017|         0007|                OGG|                 498|
+-----------------+-------------+-------------------+--------------------+
only showing top 3 rows



None

+-----------------+-------------+-------------------+--------------------+
|              _c0|          _c1|                _c2|                 _c3|
+-----------------+-------------+-------------------+--------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed ti...|
|       01/01/2018|         0005|                HNL|                 498|
|       01/01/2018|         0007|                OGG|                 501|
+-----------------+-------------+-------------------+--------------------+
only showing top 3 rows



None



df1 Count: 139359
df2 Count: 119911


259270


In [ ]:
### Read Parquet File ### (ต้องรัน Cell บนก่อน)

# Read the Parquet file into flights_df
flights_df = spark.read.parquet("AA_DFW_ALL.parquet", header=True)

# Add this line to be consistent with the question
flights_df = flights_df.withColumnRenamed("_c3", "flight_duration")

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 151


In [ ]:
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER_NAME", "name")
voter_df.show(); print("\n")

# เลือกเฉพาะแถวที่ชื่อขึ้นต้นด้วย m
print("Showing the table with name begining with M.")
voter_df.filter(voter_df["name"].like("M%")).show(5); print("\n")

print("Select specific columns: TITLE and name")
voter_df.select("TITLE", "name").show(5)

# ใช้ .filter หรือ .where ก็ได้
voter_df.filter(voter_df["TITLE"] == "Mayor").show(5)
voter_df.filter("TITLE = 'Deputy Mayor Pro Tem'").show(5)
voter_df.filter("DATE = '04/25/2018'").show(5)
voter_df.filter(~voter_df["name"].isNotNull()).show(5)
voter_df.filter(voter_df["name"].isNull()).show(5)
voter_df.filter(voter_df["TITLE"].contains('Vacant')).show(5)


+----------+-------------+-------------------+
|      DATE|        TITLE|               name|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember|       Scott Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember| Rickey D. Callahan|
|01/11/2017|Councilmember|  Jennifer S. Gates|
|04/25/2018|C

In [7]:
### Change the datetime format
from pyspark.sql.functions import year, to_date

voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
display(voter_df.printSchema())

# The code on Datacamp as of 11 Jan 2021 is incorrect on how to extract year.
# The following is how to do it properly.

voter_df = voter_df.withColumn('NewDate', to_date(voter_df['Date'], 'MM/dd/yyyy'))
display(voter_df.printSchema())
voter_df = voter_df.withColumn('Year', year(voter_df['NewDate']))
display(voter_df.printSchema())
voter_df.show(5)

root
 |-- DATE: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER_NAME: string (nullable = true)



None

root
 |-- DATE: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER_NAME: string (nullable = true)
 |-- NewDate: date (nullable = true)



None

root
 |-- DATE: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER_NAME: string (nullable = true)
 |-- NewDate: date (nullable = true)
 |-- Year: integer (nullable = true)



None

+----------+-------------+-------------------+----------+----+
|      DATE|        TITLE|         VOTER_NAME|   NewDate|Year|
+----------+-------------+-------------------+----------+----+
|02/08/2017|Councilmember|  Jennifer S. Gates|2017-02-08|2017|
|02/08/2017|Councilmember| Philip T. Kingston|2017-02-08|2017|
|02/08/2017|        Mayor|Michael S. Rawlings|2017-02-08|2017|
|02/08/2017|Councilmember|       Adam Medrano|2017-02-08|2017|
|02/08/2017|Councilmember|       Casey Thomas|2017-02-08|2017|
+----------+-------------+-------------------+----------+----+
only showing top 5 rows



In [ ]:
import pyspark.sql.functions as F

voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER_NAME", "name")
voter_df.withColumn('upper', F.upper("name")).show(5);print("\n")
voter_df.withColumn("splits", F.split("name", " ")).show(5); print("\n")

# Show the distinct name entries
print("The below table display the distinct voter's names.  Some are too long.")
voter_df.select("name").distinct().show(5, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(name) > 0 and length(name) < 20')
print("The below table removes the names that are too long.")
voter_df.select("name").distinct().show(5, truncate=False)

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('name').contains('_'))

# Show the distinct VOTER_NAME entries again
print("The below table removes the underscore from names.")
voter_df.select("name").distinct().show(5, truncate=False)

# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn("splits", F.split(voter_df["name"], '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn("first_name", voter_df["splits"].getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn("last_name", voter_df["splits"].getItem(F.size('splits') - 1)) #อย่าลืม มันเริ่มที่ 0 ถ้าขนาด array=3 ตัวสุดท้าย index=2

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show()

+----------+-------------+-------------------+-------------------+
|      DATE|        TITLE|               name|              upper|
+----------+-------------+-------------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  JENNIFER S. GATES|
|02/08/2017|Councilmember| Philip T. Kingston| PHILIP T. KINGSTON|
|02/08/2017|        Mayor|Michael S. Rawlings|MICHAEL S. RAWLINGS|
|02/08/2017|Councilmember|       Adam Medrano|       ADAM MEDRANO|
|02/08/2017|Councilmember|       Casey Thomas|       CASEY THOMAS|
+----------+-------------+-------------------+-------------------+
only showing top 5 rows



+----------+-------------+-------------------+--------------------+
|      DATE|        TITLE|               name|              splits|
+----------+-------------+-------------------+--------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|
|02/08/2017|        Mayor|Mich

In [ ]:
from pyspark.sql.functions import when
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER_NAME", "name")
voter_df.select("name", "TITLE", F.when(voter_df["TITLE"]=='Mayor', 999)).show(5)
voter_df.select("name", "TITLE", F.when(voter_df["TITLE"]=='Mayor', 999).otherwise(-1)).show(5)


+-------------------+-------------+--------------------------------------+
|               name|        TITLE|CASE WHEN (TITLE = Mayor) THEN 999 END|
+-------------------+-------------+--------------------------------------+
|  Jennifer S. Gates|Councilmember|                                  null|
| Philip T. Kingston|Councilmember|                                  null|
|Michael S. Rawlings|        Mayor|                                   999|
|       Adam Medrano|Councilmember|                                  null|
|       Casey Thomas|Councilmember|                                  null|
+-------------------+-------------+--------------------------------------+
only showing top 5 rows

+-------------------+-------------+----------------------------------------------+
|               name|        TITLE|CASE WHEN (TITLE = Mayor) THEN 999 ELSE -1 END|
+-------------------+-------------+----------------------------------------------+
|  Jennifer S. Gates|Councilmember|                

In [ ]:
# pyspark.sql.functions.when example
import pyspark.sql.functions as F
from pyspark.sql.functions import when

voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER NAME", "name")

# F.when เป็นการเพิ่มคอลัมน์ ภายใต้เงื่อนไขที่กำหนด
voter_df = voter_df.withColumn('random_val', when(voter_df["TITLE"]=="Councilmember", F.rand()))
voter_df.show(5)

# Add a column to voter_df for a voter based on their position (when สามารถ chain ต่อเนื่องได้)
voter_df = voter_df.withColumn('random_val', when(voter_df["TITLE"] == 'Councilmember', F.rand()).when(voter_df["TITLE"] == 'Mayor', 2).otherwise(0))

# Show some of the DataFrame rows
voter_df.show(5)

# Use the .filter() clause with random_val
voter_df.filter("random_val=0").show(5) # หรือจะใช้ boolean array ก็ได้


+----------+-------------+-------------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|         random_val|
+----------+-------------+-------------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|0.23697572340019812|
|02/08/2017|Councilmember| Philip T. Kingston| 0.7930585192854953|
|02/08/2017|        Mayor|Michael S. Rawlings|               null|
|02/08/2017|Councilmember|       Adam Medrano|0.32088013794978154|
|02/08/2017|Councilmember|       Casey Thomas| 0.6939513914868843|
+----------+-------------+-------------------+-------------------+
only showing top 5 rows

+----------+-------------+-------------------+------------------+
|      DATE|        TITLE|         VOTER_NAME|        random_val|
+----------+-------------+-------------------+------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|0.3469164732572111|
|02/08/2017|Councilmember| Philip T. Kingston|0.3125977806495175|
|02/08/2017|        Mayor|Michael S. Rawli

In [ ]:
# User-defined function
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import udf
from random import choice

def reverseString(s):
    return s[::-1]

def sortingCap():
    return choice(["A", "B", "C", "D"])

# เขียนจากตัวอย่างอื่น แค่เอามาให้ดูว่า udf มีตัวอย่างอะไรบ้าง
def getAvgSale(saleslist):
    totalsales, count = 0, 0
    for sale in saleslist:
        totalsales += sale[2] + sale[3] # สมมติมูลค่า sales ที่ต้องการหาอยู่ใน index ที่ 2, 3
        count += 2 # บวกทีละสอง ตามจำนวน sale ด้านบน
    return totalsales/count # เอามาคำนณค่าเฉลี่ยในแต่ละ row 

udfGetAvgSale = udf(getAvgSale, DoubleType())
# df = df.withColumn("avg_sale", udfGetAvgSale(df["sales_list"]))

# เพราะสาเหตุใดไม่ทราบ แต่ reverse ชื่อที่ยังไม่ clean ไม่ได้
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER_NAME", "name")
voter_df = voter_df.filter('length(name) > 0 and length(name) < 20')
voter_df = voter_df.filter(~ F.col('name').contains('_'))
voter_df = voter_df.withColumn("splits", F.split(voter_df["name"], '\s+'))
voter_df = voter_df.withColumn("first_name", voter_df["splits"].getItem(0))
user_df = voter_df.select("DATE", "TITLE", "name", "first_name")

# udf ต้องมี 2 positional arguments: ฟังก์ชัน และชนิดของค่าที่ return
udfReverseString = udf(reverseString, StringType())
udfReverseString2 = udf(lambda s: s[::-1], StringType())
user_df = user_df.withColumn('ReverseName', udfReverseString2(user_df["name"]))
user_df.show(5)

udfSortingCap = udf(sortingCap, StringType())
user_df.withColumn("Grade", udfSortingCap()).show(5)

+----------+-------------+-------------------+----------+-------------------+
|      DATE|        TITLE|               name|first_name|        ReverseName|
+----------+-------------+-------------------+----------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|  setaG .S refinneJ|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| notsgniK .T pilihP|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael|sgnilwaR .S leahciM|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|       onardeM madA|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|       samohT yesaC|
+----------+-------------+-------------------+----------+-------------------+
only showing top 5 rows

+----------+-------------+-------------------+----------+-------------------+-----+
|      DATE|        TITLE|               name|first_name|        ReverseName|Grade|
+----------+-------------+-------------------+----------+-------------------+-----+
|02/08/2017|Councilme

In [ ]:
### udf examples get the middle names
# These parts of codes come from previous examples.
from pyspark.sql.functions import udf
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER_NAME", "name")
voter_df = voter_df.filter('length(name) > 0 and length(name) < 20')
voter_df = voter_df.filter(~ F.col('name').contains('_'))
voter_df = voter_df.withColumn("splits", F.split(voter_df["name"], '\s+'))
voter_df = voter_df.withColumn("first_name", voter_df["splits"].getItem(0)).withColumn("last_name",  voter_df["splits"].getItem(F.size(voter_df["splits"])-1))
voter_df.show(5)
udfFirstAndMiddle = udf(lambda s: ' '.join([s[0], s[1]]), StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df["splits"]))

# Show the DataFrame
voter_df.show()



+----------+-------------+-------------------+--------------------+----------+---------+
|      DATE|        TITLE|               name|              splits|first_name|last_name|
+----------+-------------+-------------------+--------------------+----------+---------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas|
+----------+-------------+-------------------+--------------------+----------+---------+
only showing top 5 rows

+----------+-------------+-------------------+--------------------+----------+---------+---------------------+
|      DATE|        TITLE|               name|              spl

In [ ]:
### More Examples and notice how number of partitions affect pyspark.sql.functions.monotonically_increasing_id()
import pyspark.sql.functions as F

# Select all the unique council voters
df = spark.read.csv("DallasCouncilVotes.csv.gz", header=True, inferSchema=True)
voter_df = df.select("VOTER NAME").distinct()
# voter_df_single = voter_df.repartition(1) # This is costly
voter_df_single = voter_df.coalesce(1)

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID ใช้ฟังก์ชัน F.monotonically_increasing_id 
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df["ROW_ID"].desc()).show(10)

# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())
print("\nThere are %d partitions in the voter_df_single DataFrame.\n" % voter_df_single.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_single = voter_df_single.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df["ROW_ID"].desc()).show(10)
voter_df_single.orderBy(voter_df_single["ROW_ID"].desc()).show(10)


There are 36 rows in the voter_df DataFrame.

+--------------------+-------------+
|          VOTER NAME|       ROW_ID|
+--------------------+-------------+
|        Lee Kleinman|4286377361408|
| Rickey D.  Callahan|4174708211712|
|   the   final  2...|4020089389056|
|   the   final  2...|3831110828032|
|   Jennifer S. Gates|3770981285888|
| Philip T.  Kingston|3762391351296|
|  Jennifer S.  Gates|3539053051904|
|       Scott  Griggs|3478923509760|
|        Scott Griggs|3470333575168|
|   Casey  Thomas, II|3384434229248|
+--------------------+-------------+
only showing top 10 rows


There are 500 partitions in the voter_df DataFrame.


There are 1 partitions in the voter_df_single DataFrame.

+--------------------+-------------+
|          VOTER NAME|       ROW_ID|
+--------------------+-------------+
|        Lee Kleinman|4286377361408|
| Rickey D.  Callahan|4174708211712|
|   the   final  2...|4020089389056|
|   the   final  2...|3831110828032|
|   Jennifer S. Gates|3770981285888|


In [ ]:
### Caching
import pyspark.sql.functions as F
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)

# Call .cache() on the DataFrame before Action
voter_df.cache().show(2)
print("Call .cache() on the DataFrame before Action.")
print(f'The number of rows in voter_df is {voter_df.cache().count()}.\n')

# Call cache separately
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True).withColumn("ID", F.monotonically_increasing_id())
voter_df = voter_df.cache()
voter_df.show(2)

# Check cache
print(voter_df.is_cached)

# Remove cache
voter_df.unpersist()
print(voter_df.is_cached)

+----------+-------------+------------------+
|      DATE|        TITLE|        VOTER_NAME|
+----------+-------------+------------------+
|02/08/2017|Councilmember| Jennifer S. Gates|
|02/08/2017|Councilmember|Philip T. Kingston|
+----------+-------------+------------------+
only showing top 2 rows

Call .cache() on the DataFrame before Action.
The number of rows in voter_df is 44625.

+----------+-------------+------------------+---+
|      DATE|        TITLE|        VOTER_NAME| ID|
+----------+-------------+------------------+---+
|02/08/2017|Councilmember| Jennifer S. Gates|  0|
|02/08/2017|Councilmember|Philip T. Kingston|  1|
+----------+-------------+------------------+---+
only showing top 2 rows

True
False


In [ ]:
### More caching
import time
departures_df = spark.read.csv("AA_DFW_2018_Departures_Short.csv.gz", header=True, inferSchema=True)
start_time = time.time()

# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))

# Determine if departures_df is in the cache
print("Is departures_df cached?: %s" % departures_df.is_cached)
print("Removing departures_df from cache")

# Remove departures_df from the cache
departures_df.unpersist()

# Check the cache status again
print("Is departures_df cached?: %s" % departures_df.is_cached)



Counting 119910 rows took 7.821194 seconds
Counting 119910 rows again took 1.771794 seconds
Is departures_df cached?: True
Removing departures_df from cache
Is departures_df cached?: False


In [ ]:
### How to split objects

# Use OS utilities
#### split -l 10000 -d largefilename chunk- ####
# -l 10000 คือจำนวนบรรทัด
# -d ใช้ตัวเลขเป็น suffix
# largefilename คือชื่อไฟล์
# chunk- เป็น prefix

# เขียนลง parquet
df_csv =  spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True)

# สมมติมี voterdata000.csv - voterdata999.csv
# ใช้ df_csv =  spark.read.csv("voterdata*.csv", header=True, inferSchema=True)

df_csv =  df_csv.withColumnRenamed("VOTER_NAME", "name") # ไฟล์ parquet ไม่สามารถใช้ชื่อคอลัมน์มี space ได้ contains invalid character(s) among " ,;{}()\n\t=
df_csv.show(1)
df_csv =  df_csv.select("DATE", "name")
df_csv.write.parquet("data3.parquet")
df_pq = spark.read.parquet("data3.parquet")
df_pq.show(4)

+----------+-------------+-----------------+
|      DATE|        TITLE|             name|
+----------+-------------+-----------------+
|02/08/2017|Councilmember|Jennifer S. Gates|
+----------+-------------+-----------------+
only showing top 1 row

+----------+-------------------+
|      DATE|               name|
+----------+-------------------+
|02/08/2017|  Jennifer S. Gates|
|02/08/2017| Philip T. Kingston|
|02/08/2017|Michael S. Rawlings|
|02/08/2017|       Adam Medrano|
+----------+-------------------+
only showing top 4 rows



In [ ]:
### Reading Spark configurations

# Name of the Spark application instance
app_name = spark.conf.get("spark.app.name")
  
# ถ้าจะเขียน config ให้เขียน app_name = spark.conf.set("pyspark-shell")

# Driver TCP port
driver_tcp_port = spark.conf.get("spark.driver.port")

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions, "\n\n")

# Set Spark config

# Store the number of partitions in variable.  Question asks to set distinct rows
departures_df = spark.read.csv('AA_DFW_2018_Departures_Short.csv.gz').distinct()
before = departures_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv('AA_DFW_2017_Departures_Short.csv.gz').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

Name: pyspark-shell
Driver TCP port: 35617
Number of partitions: 200 


Partition count before change: 200
Partition count after change: 500


In [ ]:
### Performance Improvements using F.broadcast in join method
### Tips: use F.broadcast(smaller_df)
### ถ้า smaller_df เล็กกว่ามากๆ อาจจะไม่ต้องใช้ broadcast

import pyspark.sql.functions as F
voter_df = spark.read.csv("DallasCouncilVoters.csv.gz", header=True, inferSchema=True).select("VOTER_NAME").distinct()

# To understand performance implications of Spark, use .explain method on a spark dataframe
voter_df.explain()

# Join the flights_df and aiports_df DataFrames
flights_df = spark.read.csv("AA_DFW_2018_Departures_Short.csv.gz",  header=True, inferSchema=True)
airports_df = spark.read.csv("airportnames.txt.gz",  header=True, inferSchema=True)
normal_df = flights_df.join(airports_df,flights_df["Destination Airport"]==airports_df["IATA"] )

# Show the query plan
normal_df.explain()

broadcast_df = flights_df.join(F.broadcast(airports_df), flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.explain()

start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = normal_df.count()
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))



== Physical Plan ==
*(2) HashAggregate(keys=[VOTER_NAME#1032], functions=[])
+- Exchange hashpartitioning(VOTER_NAME#1032, 500), true, [id=#1527]
   +- *(1) HashAggregate(keys=[VOTER_NAME#1032], functions=[])
      +- FileScan csv [VOTER_NAME#1032] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/content/DallasCouncilVoters.csv.gz], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<VOTER_NAME:string>


== Physical Plan ==
*(2) BroadcastHashJoin [Destination Airport#1055], [IATA#1078], Inner, BuildRight
:- *(2) Project [Date (MM/DD/YYYY)#1053, Flight Number#1054, Destination Airport#1055, Actual elapsed time (Minutes)#1056]
:  +- *(2) Filter isnotnull(Destination Airport#1055)
:     +- FileScan csv [Date (MM/DD/YYYY)#1053,Flight Number#1054,Destination Airport#1055,Actual elapsed time (Minutes)#1056] Batched: false, DataFilters: [isnotnull(Destination Airport#1055)], Format: CSV, Location: InMemoryFileIndex[file:/content/AA_DFW_2018_Departures_Sh

In [ ]:
### Data Pipelines

# Import the data to a DataFrame
departures_df = spark.read.csv("AA_DFW_2015_Departures_Short.csv.gz", header=True)

# Remove any duration of 0
departures_df = departures_df.filter(departures_df["Actual elapsed time (Minutes)"] > 0)
# departures_df = departures_df.filter(departures_df[3] > 0) ก็ได้เหมือนกัน

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
departures_df.write.json("output.json", mode='overwrite')


In [ ]:
### Removng commented lines

# df = spark.read.csv("datafile.csv", comment="#") เอาแถวที่มีเครื่องหมาย # ออกไป

import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, DoubleType


# Import the file to a DataFrame and perform a row count
annotations_df = spark.read.csv('annotations.csv.gz', sep="|")
full_count = annotations_df.count()

print("The initial data looks as below:")
annotations_df.show(5)

# Count the number of rows beginning with '#'
comment_count = annotations_df.where(col('_c0').startswith('#')).count()
print("The commented lines looks like below:")
annotations_df.filter(col('_c0').startswith('#')).show(5)

# Import the file to a new DataFrame, without commented rows
no_comments_df = spark.read.csv('annotations.csv.gz', sep="|", comment='#')

# Count the new DataFrame and verify the difference is as expected
no_comments_count = no_comments_df.count()
print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count), "\n")

### Removing invalid rows ###

# Split _c0 on the tab character and store the list in a variable
tmp_fields = F.split(annotations_df['_c0'], "\t")
print("F.split(annotations_df['_c0'],'\\t') is of type", type(F.split(annotations_df['_c0'], '\t')), "\n")

# Create the colcount column on the DataFrame
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter("colcount < 5")

# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (no_comments_count, final_count))
print("The current data looks like this:")
annotations_df_filtered.show(5)

### Splitting into columns ###

# The question use this DataFrame.  It didn't use the filtered one.
annotations_df = annotations_df.filter("colcount >= 5")

# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df['_c0'], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width', split_cols.getItem(2))
split_df = split_df.withColumn('height', split_cols.getItem(3))

# Add split_cols as a column
split_df = split_df.withColumn('split_cols', split_cols)
print("Now the dataframe looks like this")
split_df.show(5)


### Using F.udf

# First we need to know where the numeber 4 comes from
print("First we need to know where the numeber 4 comes from.")
print("Using the keyword argument 'truncate=False' in .show() method reveals the details.")
split_df.select("split_cols").show(15, truncate=False)
print("Notice that the breed names start at index 4 onward.")
print("Some rows have more than one dog information. \n")

def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))
udfRetriever_lambda = F.udf(lambda x: x[4:], ArrayType(StringType()))

# Create a new column using your UDF
split_df = split_df.withColumn('dog_list', udfRetriever(split_df["split_cols"], split_df["colcount"]))
split_df_lambda = split_df.withColumn('dog_list', udfRetriever_lambda(split_df["split_cols"]))

# Remove the original column, split_cols, and the colcount
split_df = split_df.drop('_c0').drop('colcount').drop('split_cols')
split_df_lambda = split_df.drop('_c0').drop('colcount').drop('split_cols')
print("The final data look like this")
split_df.show(5)

# Check if the lambda version of udf returns to same dataframe
print(f"Does the lambda version of udf return to same dataframe?: {(split_df.toPandas()).equals(split_df_lambda.toPandas())} ")


The initial data looks as below:
+--------------------+
|                 _c0|
+--------------------+
|025865917	n023521...|
|022684404	n029380...|
|021267273	n022910...|
|02110627	n0211062...|
|02093754	n0209375...|
+--------------------+
only showing top 5 rows

The commented lines looks like below:
+------+
|   _c0|
+------+
| # 339|
| # 968|
|# 1321|
|# 1251|
|# 1402|
+------+
only showing top 5 rows

Full count: 32794
Comment count: 1416
Remaining count: 31378 

F.split(annotations_df['_c0'],'\t') is of type <class 'pyspark.sql.column.Column'> 

Initial count: 31378
Final count: 12214
The current data looks like this:
+--------------------+--------+
|                 _c0|colcount|
+--------------------+--------+
|025865917	n023521...|       2|
|022684404	n029380...|       2|
|021267273	n022910...|       2|
|023200662	n023050...|       2|
|028666219	n025734...|       2|
+--------------------+--------+
only showing top 5 rows

Now the dataframe looks like this
+--------------------+

In [ ]:
### Validate rows via join ###

valid_folders_df = spark.read.csv("valid_folders.txt")
print("The valid folders data look like this: ")
valid_folders_df.show(5)

# Rename the column in valid_folders_df
valid_folders_df = valid_folders_df.withColumnRenamed("_c0", "folder")

# Count the number of rows in split_df
split_count = split_df.count()

# Join the DataFrames
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

# Create a DataFrame containing the invalid rows
invalid_df = split_df.join(joined_df, on='folder', how='leftOuter')

# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# Determine the number of distinct folder rows removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)

The valid folders data look like this: 
+--------+
|     _c0|
+--------+
|02085620|
|02085782|
|02085936|
|02086079|
|02086240|
+--------+
only showing top 5 rows

Before: 20580
After: 19956
 split_df:	20580
 joined_df:	19956
 invalid_df: 	3496880
117 distinct invalid folders found


In [ ]:
### Dog Parsing
# Select the dog details and show 10 untruncated rows
print(joined_df.select("dog_list").show(10, truncate=False))

# Define a schema type for the details in the dog list
DogType = StructType([
	StructField("breed", StringType(), False),
    StructField("start_x", IntegerType(), False),
    StructField("start_y", IntegerType(), False),
    StructField("end_x", IntegerType(), False),
    StructField("end_y", IntegerType(), False)])

# Create a function to return the number and type of dogs as a tuple
def dogParse(doglist):
  dogs = []
  for dog in doglist:
    (breed, start_x, start_y, end_x, end_y) = dog.split(',')
    dogs.append((breed, int(start_x), int(start_y), int(end_x), int(end_y)))
  return dogs

# Create a UDF
udfDogParse = F.udf(dogParse, ArrayType(DogType))

# Use the UDF to list of dogs and drop the old column
joined_df = joined_df.withColumn('dogs', udfDogParse('dog_list')).drop('dog_list')

# Show the number of dogs in the first 10 rows
joined_df.select(F.size('dogs')).show(10)

# Define a UDF to determine the number of pixels per image
def dogPixelCount(doglist):
  totalpixels = 0
  for dog in doglist:
    totalpixels += (dog[3] - dog[1]) * (dog[4] - dog[2])
  return totalpixels

# Define a UDF for the pixel count
udfDogPixelCount = F.udf(dogPixelCount, IntegerType())
joined_df = joined_df.withColumn("dog_pixels", udfDogPixelCount("dogs"))

# Create a column representing the percentage of pixels
joined_df = joined_df.withColumn('dog_percent', (joined_df["dog_pixels"] / (joined_df["width"]*joined_df["height"])) * 100)

# Show the first 10 annotations with more than 60% dog
joined_df.filter("dog_percent > 60").show(10)

+----------------------------------+
|dog_list                          |
+----------------------------------+
|[affenpinscher,0,9,173,298]       |
|[Border_terrier,73,127,341,335]   |
|[kuvasz,0,0,499,327]              |
|[Great_Pyrenees,124,225,403,374]  |
|[schipperke,146,29,416,309]       |
|[groenendael,168,0,469,374]       |
|[Bedlington_terrier,10,12,462,332]|
|[Lhasa,39,1,499,373]              |
|[Kerry_blue_terrier,17,16,300,482]|
|[vizsla,112,93,276,236]           |
+----------------------------------+
only showing top 10 rows

None
+----------+
|size(dogs)|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 10 rows

+--------+---------------+-----+------+--------------------+----------+-----------------+
|  folder|       filename|width|height|                dogs|dog_pixels|      dog_percent|
+--------+---------------+-----+------+--------------------+---

+----------+
|size(dogs)|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 10 rows



In [ ]:
spark.version

'3.0.1'

In [ ]:
import sys; sys.version_info

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)